# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('homepage_actions.csv')
# Count the number of viewers and clickers
viewers = df[df['action'] == 'view'].shape[0]
clickers = df[df['action'] == 'click'].shape[0]

# Print the results
print(f"Number of viewers: {viewers}")
print(f"Number of clickers: {clickers}")
print(f"Number of viewers who also clicked: {df[df['action'] == 'click']['id'].nunique()}")

# Check for anomalies
anomalies = df[df['action'] == 'click'][~df['id'].isin(df[df['action'] == 'view']['id'].unique())]
print(f"Number of anomalies: {anomalies.shape[0]}")

# Check for overlap between control and experiment groups
control_ids = df[df['group'] == 'control']['id'].unique()
experiment_ids = df[df['group'] == 'experiment']['id'].unique()
overlap = set(control_ids) & set(experiment_ids)
print(f"Number of users in both groups: {len(overlap)}")

Number of viewers: 6328
Number of clickers: 1860
Number of viewers who also clicked: 1860
Number of anomalies: 0
Number of users in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [12]:
# Conduct a two-sample t-test
from scipy.stats import ttest_ind
# Calculate the click-through rates for the control and experiment groups
control_df = df[df['group'] == 'control']
experiment_df = df[df['group'] == 'experiment']
control_ctr = control_df[control_df['action'] == 'click'].shape[0] / control_df[control_df['action'] == 'view'].shape[0]
experiment_ctr = experiment_df[experiment_df['action'] == 'click'].shape[0] / experiment_df[experiment_df['action'] == 'view'].shape[0]
t_stat, p_value = ttest_ind(control_df[control_df['action'] == 'click']['id'].unique(),
                            experiment_df[experiment_df['action'] == 'click']['id'].unique())
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

t-statistic: -0.5828
p-value: 0.5601


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [7]:
expected_clicks = experiment_df[experiment_df['action'] == 'view'].shape[0] * control_ctr
print(f"Expected clicks in experiment group: {expected_clicks:.2f}")

Expected clicks in experiment group: 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [8]:
# Calculate the actual number of clicks in the experiment group
actual_clicks = experiment_df[experiment_df['action'] == 'click'].shape[0]

# Calculate the variance for a binomial variable with n observations
n = experiment_df[experiment_df['action'] == 'view'].shape[0]
p = control_ctr
variance = n * p * (1 - p)

# Calculate the standard deviation
std_dev = variance ** 0.5

# Calculate the z-score
z_score = (actual_clicks - expected_clicks) / std_dev
print(f"Z-score: {z_score:.2f}")

Z-score: 3.66


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
from scipy.stats import norm
p_value = 1 - norm.cdf(z_score)
print(f"P-value: {p_value:.4f}")

P-value: 0.0001


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **These two p-values are very different, indicating that the two tests lead to different conclusions. a p-value of 0.001 would suggest strong evidence that the experimental homepage is more effective than the control homepage, while a p-value of 0.5601 would suggest that there is not a statistically significant difference in click-through rates between the two homepages.**


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.